In [ ]:
import pandas as pd
import numpy as np
import random

csv1_path = "data.csv"  
csv2_path = "data_blip2SDXL.csv"  

df1 = pd.read_csv(csv1_path)
df2 = pd.read_csv(csv2_path)

# Paso 1: Filtrar las columnas necesarias
df1 = df1[['file_name', 'description']]
df2 = df2[['file_name', 'blip2']]

# Paso 2: Normalizar los nombres de archivo en el segundo CSV
df2['file_name'] = df2['file_name'].str.replace('_generated', '', regex=False)

# Paso 3: Hacer el match usando un merge OUTER para identificar registros sin coincidencia
merged_df = pd.merge(df1, df2, on='file_name', how='outer', indicator=True)

# Paso 4: Identificar registros sin coincidencia
df1_only = merged_df[merged_df['_merge'] == 'left_only']
df2_only = merged_df[merged_df['_merge'] == 'right_only']

print(f"Registros en el primer CSV sin coincidencia en el segundo CSV: {len(df1_only)}")
print(f"Registros en el segundo CSV sin coincidencia en el primer CSV: {len(df2_only)}")

if len(df1_only) > 0:
    print("Registros únicos en el primer CSV:")
    print(df1_only[['file_name']])

if len(df2_only) > 0:
    print("Registros únicos en el segundo CSV:")
    print(df2_only[['file_name']])

# Paso 5: Procesar solo los registros que tienen coincidencia (inner merge)
matched_df = pd.merge(df1, df2, on='file_name', how='inner')

# Paso 6: Extraer el género (carpeta) de cada file_name
matched_df['genre'] = matched_df['file_name'].apply(lambda x: x.split('/')[0] if '/' in x else "unknown")

# Paso 7: Generar las columnas requeridas para imágenes originales y generadas
matched_df['original_image'] = 'resize768/' + matched_df['file_name']
matched_df['generated_image'] = 'SDXL/' + matched_df['file_name'].str.replace('.jpg', '_generated.jpg', regex=False)
matched_df['description_original_image'] = matched_df['description']
matched_df['description_generated_image'] = matched_df['blip2']

# Paso 8: Crear un diccionario para organizar las filas por género
genre_dict = {}
for genre in matched_df['genre'].unique():
    genre_dict[genre] = matched_df[matched_df['genre'] == genre].index.tolist()

# Paso 9: Seleccionar aleatoriamente imágenes negativas de diferentes géneros sin repetir
df1_copy = df1.copy()
df1_copy['genre'] = df1_copy['file_name'].apply(lambda x: x.split('/')[0] if '/' in x else "unknown")

matched_df['negative_image'] = None
matched_df['description_negative_image'] = None

# Conjunto para rastrear imágenes negativas ya utilizadas
used_negative_images = set()

# Para cada fila en matched_df
for idx, row in matched_df.iterrows():
    current_genre = row['genre']
    
    # Encontrar todos los géneros diferentes al actual
    other_genres = [g for g in genre_dict.keys() if g != current_genre]
    
    # Si no hay otros géneros disponibles, usar cualquier género (caso improbable)
    if not other_genres:
        other_genres = list(genre_dict.keys())
    
    # Seleccionar aleatoriamente un género diferente
    random_genre = random.choice(other_genres)
    
    # Intentar encontrar una imagen no utilizada
    potential_indices = [i for i in genre_dict[random_genre] if i not in used_negative_images]
    
    # Si no hay imágenes disponibles en este género, intentar con otro
    attempts = 0
    while not potential_indices and attempts < len(other_genres):
        random_genre = random.choice(other_genres)
        potential_indices = [i for i in genre_dict[random_genre] if i not in used_negative_images]
        attempts += 1
    
    # Si todavía no tenemos opciones, usar cualquiera que no haya sido usada
    if not potential_indices:
        all_unused = [i for i in range(len(matched_df)) if i not in used_negative_images and i != idx]
        if all_unused:
            selected_idx = random.choice(all_unused)
        else:
            # Si todas han sido usadas, simplemente elegir una diferente a la actual
            all_other = [i for i in range(len(matched_df)) if i != idx]
            selected_idx = random.choice(all_other) if all_other else 0
    else:
        selected_idx = random.choice(potential_indices)
    
    # Marcar esta imagen como utilizada
    used_negative_images.add(selected_idx)
    
    # Asignar la imagen negativa y su descripción
    neg_file_name = matched_df.iloc[selected_idx]['file_name']
    matched_df.at[idx, 'negative_image'] = 'resize768/' + neg_file_name
    
    # Buscar la descripción original en df1_copy
    neg_description = df1_copy[df1_copy['file_name'] == neg_file_name]['description'].values[0]
    matched_df.at[idx, 'description_negative_image'] = neg_description

# Paso 10: Seleccionar solo las columnas finales requeridas
result_df = matched_df[['original_image', 'generated_image', 'negative_image', 
                         'description_original_image', 'description_generated_image', 'description_negative_image']]

# Guardar el resultado en un nuevo CSV
output_path = "archivo_resultado_con_negativas.csv"
result_df.to_csv(output_path, index=False)
print("Proceso completado. El archivo resultante se ha guardado en:", output_path)

Registros en el primer CSV sin coincidencia en el segundo CSV: 0
Registros en el segundo CSV sin coincidencia en el primer CSV: 0
Proceso completado. El archivo resultante se ha guardado en: archivo_resultado_con_negativas.csv


In [4]:
import pandas as pd
import os
from config import Config
csv_path = "data.csv"
base_folder = Config.IMAGES_DIR
df = pd.read_csv(csv_path)

missing_original = []
missing_generated = []
missing_negative = []

total_rows = len(df)
print(f"Verificando {total_rows} filas de imágenes...")

for idx, row in df.iterrows():
    original_path = os.path.join(base_folder, row['original_image'])
    generated_path = os.path.join(base_folder, row['generated_image'])
    negative_path = os.path.join(base_folder, row['negative_image'])
    
    if not os.path.isfile(original_path):
        missing_original.append(original_path)
    
    if not os.path.isfile(generated_path):
        missing_generated.append(generated_path)
    
    if not os.path.isfile(negative_path):
        missing_negative.append(negative_path)
    
    if (idx + 1) % 100 == 0:
        print(f"Procesadas {idx + 1}/{total_rows} filas...")

print("\n--- RESULTADOS DE LA VERIFICACIÓN ---")
print(f"Total de imágenes originales no encontradas: {len(missing_original)}")
print(f"Total de imágenes generadas no encontradas: {len(missing_generated)}")
print(f"Total de imágenes negativas no encontradas: {len(missing_negative)}")

if missing_original:
    print("\nImágenes originales no encontradas:")
    for path in missing_original:
        print(f"- {path}")

if missing_generated:
    print("\nImágenes generadas no encontradas:")
    for path in missing_generated:
        print(f"- {path}")

if missing_negative:
    print("\nImágenes negativas no encontradas:")
    for path in missing_negative:
        print(f"- {path}")

Verificando 81444 filas de imágenes...
Procesadas 100/81444 filas...
Procesadas 200/81444 filas...
Procesadas 300/81444 filas...
Procesadas 400/81444 filas...
Procesadas 500/81444 filas...
Procesadas 600/81444 filas...
Procesadas 700/81444 filas...
Procesadas 800/81444 filas...
Procesadas 900/81444 filas...
Procesadas 1000/81444 filas...
Procesadas 1100/81444 filas...
Procesadas 1200/81444 filas...
Procesadas 1300/81444 filas...
Procesadas 1400/81444 filas...
Procesadas 1500/81444 filas...
Procesadas 1600/81444 filas...
Procesadas 1700/81444 filas...
Procesadas 1800/81444 filas...
Procesadas 1900/81444 filas...
Procesadas 2000/81444 filas...
Procesadas 2100/81444 filas...
Procesadas 2200/81444 filas...
Procesadas 2300/81444 filas...
Procesadas 2400/81444 filas...
Procesadas 2500/81444 filas...
Procesadas 2600/81444 filas...
Procesadas 2700/81444 filas...
Procesadas 2800/81444 filas...
Procesadas 2900/81444 filas...
Procesadas 3000/81444 filas...
Procesadas 3100/81444 filas...
Procesada